In [ ]:
import os
import glob
import data
import numpy as np
import torch
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm 
from utils.distmat import *
from utils.evaluation import *
from hitl import *

In [ ]:
paths = glob.glob("match_matrix/*.npy")
paths

In [ ]:
path = paths[0]
matches = np.load(path)
matches = torch.as_tensor(matches)
matches.shape

In [ ]:
match = re_path.match(path)
match.group(1)

In [ ]:
re_path = re.compile(".*(bot_[a-z]*_basic)_([a-z\_]+).npy")
match = re_path.match(path)
key = match.group(1)
method = match.group(2)

In [ ]:
def get_results(path, key):
    output = data.load_output(key)
    qf = np.array(output["qf"])
    gf = np.array(output["gf"])
    q_pids = np.array(output["q_pids"])
    g_pids = np.array(output["g_pids"])
    q_camids = np.array(output["q_camids"])
    g_camids = np.array(output["g_camids"])
    
    max_rank = 50
    device = None
    q, g = len(q_pids), len(g_pids)
    keep = torch.ones(q, g, dtype=bool)
    kept = keep.cumsum(dim=1)
    
    matches = np.load(path)
    matches = torch.as_tensor(matches)
    matches.shape

    valid_matches = matches * keep
    valid_query = (valid_matches.sum(dim=1) > 0)  # at least one matchable (== matched) gallery image
    assert (valid_matches.sum() != 0)  # error: all query identities do not appear in gallery

    final_rank_positions = (valid_matches * torch.arange(1, g + 1, device=device)).argmax(dim=1)
    final_rank_valid = kept[torch.arange(q, device=device), final_rank_positions]
    all_INP = valid_matches.sum(dim=1).float() / final_rank_valid.float()

    # `kept` is analogous to index within only-valid instances
    cum_precision = valid_matches.cumsum(dim=1).float() / kept.float()
    cum_precision[cum_precision.isnan()] = 1
    all_AP = (cum_precision * valid_matches).sum(dim=1) / valid_matches.sum(dim=1)

    # Compute CMC (need to go query-by-query) (assume that at least 10% are valid)
    buffer = 10
    keep = keep[:, :max_rank * buffer]
    matches = matches[:, :max_rank * buffer]
    all_cmc = []
    for i in range(q):
        mc = matches[i][keep[i]][:50]
        if len(mc) < max_rank:
            raise AssertionError("Not enough matching galleries. Consider higher `buffer` value.")
        cmc = mc[:max_rank].cumsum(dim=0)
        # E.g., 0 1 x x x x ... to 0 1 1 1 1 1 ...
        cmc[cmc > 1] = 1
        all_cmc.append(cmc)

    all_cmc = torch.stack(all_cmc).float()
    all_cmc = all_cmc.sum(dim=0) / valid_query.float().sum()

    mAP = all_AP[valid_query].mean()
    mINP = all_INP[valid_query].mean()

    matches_by_rank = valid_matches.int().sum(dim=0)
    cum_matches = matches_by_rank.cumsum(dim=0)
    total = torch.arange(1, g + 1).float() * q
    recall = cum_matches.float() / matches_by_rank.float().sum()
    precision = cum_matches.float() / total.float()
    
    return mAP, mINP, recall, precision, matches_by_rank.sum()

In [ ]:
def get_baseline_results(key, rerank=False):
    output = data.load_output(key)
    qf = np.array(output["qf"])
    gf = np.array(output["gf"])
    q_pids = np.array(output["q_pids"])
    g_pids = np.array(output["g_pids"])
    q_camids = np.array(output["q_camids"])
    g_camids = np.array(output["g_camids"])
    q, g = len(q_pids), len(g_pids)
    
    if rerank:
        distmat_all = compute_inner_distmat(torch.cat([torch.as_tensor(qf), torch.as_tensor(gf)], dim=0))
        distmat = rerank_distmat(distmat_all, q)
    else:
        distmat = compute_distmat(torch.as_tensor(qf), torch.as_tensor(gf))

    indices = distmat.argsort()
    matches = (g_pids[indices] == q_pids.reshape(-1, 1))
    matches = torch.as_tensor(matches)

    max_rank = 50
    device = None
    keep = torch.ones(q, g, dtype=bool)
    kept = keep.cumsum(dim=1)

    valid_matches = matches * keep
    valid_query = (valid_matches.sum(dim=1) > 0)  # at least one matchable (== matched) gallery image
    assert (valid_matches.sum() != 0)  # error: all query identities do not appear in gallery

    final_rank_positions = (valid_matches * torch.arange(1, g + 1, device=device)).argmax(dim=1)
    final_rank_valid = kept[torch.arange(q, device=device), final_rank_positions]
    all_INP = valid_matches.sum(dim=1).float() / final_rank_valid.float()

    # `kept` is analogous to index within only-valid instances
    cum_precision = valid_matches.cumsum(dim=1).float() / kept.float()
    cum_precision[cum_precision.isnan()] = 1
    all_AP = (cum_precision * valid_matches).sum(dim=1) / valid_matches.sum(dim=1)

    # Compute CMC (need to go query-by-query) (assume that at least 10% are valid)
    buffer = 10
    keep = keep[:, :max_rank * buffer]
    matches = matches[:, :max_rank * buffer]
    all_cmc = []
    for i in range(q):
        mc = matches[i][keep[i]][:50]
        if len(mc) < max_rank:
            raise AssertionError("Not enough matching galleries. Consider higher `buffer` value.")
        cmc = mc[:max_rank].cumsum(dim=0)
        # E.g., 0 1 x x x x ... to 0 1 1 1 1 1 ...
        cmc[cmc > 1] = 1
        all_cmc.append(cmc)

    all_cmc = torch.stack(all_cmc).float()
    all_cmc = all_cmc.sum(dim=0) / valid_query.float().sum()

    mAP = all_AP[valid_query].mean()
    mINP = all_INP[valid_query].mean()

    matches_by_rank = valid_matches.int().sum(dim=0)
    cum_matches = matches_by_rank.cumsum(dim=0)
    total = torch.arange(1, g + 1).float() * q
    recall = cum_matches.float() / matches_by_rank.float().sum()
    precision = cum_matches.float() / total.float()
    
    return mAP, mINP, recall, precision, matches_by_rank.sum()

In [ ]:
#scores = {}
#pr = {}
for path in tqdm(paths):
    match = re_path.match(path)
    key = match.group(1)
    method = match.group(2)
    record = "{}_{}".format(key, method)
    mAP, mINP, recall, precision, matches = get_results(path, key)
    scores[record] = {
        "mAP": float(mAP),
        "mINP": float(mINP),
    }
    print(key, method, matches)
    pr[record] = (recall, precision)
    fig = plt.figure(figsize=(8, 6))
    fig.suptitle("ROC Curve [{}] [{}]".format(key, method))
    ax = fig.subplots()
    ax.plot(recall, precision)
    fig.savefig("Roc Curve [{}] [{}]".format(key, method))

In [ ]:
for key in ["bot_market_basic", "bot_duke_basic"]:
    method = "baseline"
    record = "{}_{}".format(key, method)
    mAP, mINP, recall, precision, matches = get_baseline_results(key, False)
    scores[record] = {
        "mAP": float(mAP),
        "mINP": float(mINP),
    }
    pr[record] = (recall, precision)
    print(key, method, matches)
    fig = plt.figure(figsize=(8, 6))
    fig.suptitle("ROC Curve [{}] [{}]".format(key, method))
    ax = fig.subplots()
    ax.plot(recall, precision)
    fig.savefig("Roc Curve [{}] [{}]".format(key, method))
    
    method = "baseline_rerank"
    record = "{}_{}".format(key, method)
    mAP, mINP, recall, precision, matches = get_baseline_results(key, True)
    scores[record] = {
        "mAP": float(mAP),
        "mINP": float(mINP),
    }
    pr[record] = (recall, precision)
    print(key, method, matches)
    fig = plt.figure(figsize=(8, 6))
    fig.suptitle("ROC Curve [{}] [{}]".format(key, method))
    ax = fig.subplots()
    ax.plot(recall, precision)
    fig.savefig("Roc Curve [{}] [{}]".format(key, method))

In [ ]:
list(pr.keys())

In [ ]:
pr["bot_duke_basic_baseline"]

In [ ]:
pr["bot_duke_basic_ne_min"]

In [ ]:
fig = plt.figure(figsize=(12, 10))
fig.suptitle("ROC Curve [{}]".format("market"))
ax = fig.subplots()
re_key= re.compile("(bot_[a-z]*_basic)_([a-z\_]+)")
lines = []
methods = []
for key, value in pr.items():
    match = re_key.match(key)
    dataset= match.group(1)
    method = match.group(2)
    if "market" in dataset:
        print(dataset, method)
        r, p = value
        line = ax.plot(r, p, label=method)
        lines.append(line)
        methods.append(method)
ax.legend()
fig.savefig("Roc Curve [{}].jpg".format("market", method), dpi=320)

In [ ]:
pr[""]

In [ ]:
fig = plt.figure(figsize=(12, 10))
fig.suptitle("ROC Curve [{}]".format("duke"))
ax = fig.subplots()
re_key= re.compile("(bot_[a-z]*_basic)_([a-z\_]+)")
lines = []
methods = []
for key, value in pr.items():
    match = re_key.match(key)
    dataset= match.group(1)
    method = match.group(2)
    if "duke" in dataset:
        print(dataset, method)
        r, p = value
        line = ax.plot(r, p, label=method)
        lines.append(line)
        methods.append(method)
ax.legend()
fig.savefig("Roc Curve [{}].jpg".format("duke", method), dpi=320)

In [ ]:
pr_dict = {}
re_key= re.compile("(bot_[a-z]*_basic)_([a-z\_]+)")
for key, val in pr.items():
    if
    match = re_key.match(key)
    dataset= match.group(1)
    method = match.group(2)
    print(dataset, method)

In [ ]:
df_scores = pd.DataFrame(scores).T
df_scores = df_scores.sort_index()
df_scores.to_csv("scores.csv")
df_scores